In [32]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import math
import folium
from folium import GeoJson, LinearColormap
from folium.features import DivIcon
from folium.plugins import HeatMap
import warnings
import branca

warnings.filterwarnings('ignore')

In [33]:
# try and reduce the amount of times you run this itll take forever.
##### only need this cell if you are wanting to check against other map features see bash script to decide what features you want
gdf = gpd.read_file('.\output.geojson')
boundaries = gpd.read_file('.\\boundaries.geojson')

# manually fix names set() ^ set()
boundaries.loc[boundaries.name == 'Juan de Fuca Electoral Area', 'name'] = 'Juan de Fuca'
boundaries.loc[boundaries.name == 'Salt Spring Island Electoral Area', 'name'] = 'Salt Spring Island'
boundaries.loc[boundaries.name == 'Southern Gulf Islands Electoral Area', 'name'] = 'Southern Gulf Islands'

def remove_times():
    # use for error in folio TypeError: Object of type Timestamp is not JSON serializable
    drop_ls = []
    for c in gdf.columns:
        if ("time" in c or "date" in c or "hour" in c):
            print(c)
            drop_ls.append(c)

    return gdf.drop(drop_ls, axis=1)

gdf = remove_times() # only use if you have an error with folium Geo.Json


check_date
survey:date


In [34]:
startDt = pd.read_csv(".\May2023.csv")
endDt = pd.read_csv(".\May2024.csv")

In [35]:
pd.set_option("display.max_rows", None)
endDt.host_name.value_counts()['Carol Ann']
startDt.host_name.value_counts()

host_name
Elite Vacation Homes                 71
Co-Hosts                             50
Iris Properties                      49
Angela                               48
Victoria Prime Services              44
Don                                  38
Your Host Marketing                  36
Jan                                  30
John                                 29
Rob&Jen                              29
Robyn                                26
Laura                                25
HFP Marketing                        24
David                                24
Emr                                  24
Radie                                23
Michael                              23
Michelle                             23
Anna                                 22
Jennifer                             21
Melissa                              20
Martin                               19
Kelly                                19
Rick                                 19
Heather                       

In [36]:
# get the center of all unique munis

def get_weight(group):
    val = endDt.neighbourhood_group.value_counts()[group] - startDt.neighbourhood_group.value_counts()[group]
    return val / max(startDt.neighbourhood_group.value_counts()[group], endDt.neighbourhood_group.value_counts()[group])

def get_center_loc(group, id=None, res_dic=None):
    # group is a data frame
    # source http://www.geomidpoint.com/example.html
    # gets the center lat lng
    # assuming the earth is perfectly round
    rads_lat = np.radians(group.latitude)
    rads_lon = np.radians(group.longitude)
    
    # Convert lat/lon (radians) to Cartesian coordinates for each location.
    X = np.cos(rads_lat) * np.cos(rads_lon)
    Y = np.cos(rads_lat) * np.sin(rads_lon)
    Z = np.sin(rads_lat)

    #find average x, y, z coords
    x = X.mean()
    y = Y.mean()
    z = Z.mean()

    # Convert average x, y, z coordinate to latitude and longitude.
    lng = math.atan2(y, x)
    hyp = np.sqrt(x*x + y*y)
    lat = math.atan2(z, hyp)
    if res_dic != None:
        res_dic[id] = [np.degrees(lat), np.degrees(lng), len(group.id.tolist())-endDt.neighbourhood_group.value_counts()[id], get_weight(id)]
    else:
        return (np.degrees(lat), np.degrees(lng),)



                          
result_start = {}
startDt.groupby(['neighbourhood_group']).apply(lambda group: get_center_loc(group=group, id=group.name, res_dic=result_start))


startDt['location'] = startDt.apply(lambda row: [row.latitude, row.longitude, get_weight(row.neighbourhood_group)], axis=1)
endDt['location'] = endDt.apply(lambda row: [row['latitude'], row['longitude'], get_weight(row.neighbourhood_group)], axis=1)

In [37]:
startDt.location.head()

0    [48.42128, -123.33932, -0.32494608195542773]
1    [48.42515, -123.33977, -0.32494608195542773]
2    [48.41333, -123.37065, -0.32494608195542773]
3    [48.42151, -123.36383, -0.32494608195542773]
4     [48.86282, -123.49976, 0.06876790830945559]
Name: location, dtype: object

In [38]:
# break up all the startDt data into single owners and multi owners
start_count = startDt.host_id.value_counts()
multi_start = startDt[startDt.host_id.isin(start_count.index[start_count.gt(1)])]
single_start = startDt[startDt.host_id.isin(start_count.index[start_count.lt(2)])]

# break up all endDt data into multi owners and single owners
end_count = endDt.host_id.value_counts()
multi_end = endDt[endDt.host_id.isin(end_count.index[end_count.gt(1)])]
single_end = endDt[endDt.host_id.isin(end_count.index[end_count.lt(2)])]

In [39]:
# get mean price grouped by host_id
# values tracking multi_start, multi_end, single_start, single_end, singleDt, endDt
multi_start['mean_price'] = multi_start.groupby(['host_id']).price.transform('mean')
multi_end['mean_price'] = multi_end.groupby(['host_id']).price.transform('mean')

In [40]:
    
# get real center for group listings
# this function could be cleaned up
def get_group_center(group, center):
    i = 0
    for index, row in group.iterrows():
        if i == 0:
            closest = (row.latitude, row.longitude)
            i += 1
        if abs(row.latitude - center[row.host_name][0] + row.longitude - center[row.host_name][1]) < abs(closest[0]-center[row.host_name][0] + closest[1]-center[row.host_name][1]):
            closest = [row.latitude, row.longitude]
    return closest

# find closest to center listing
def find_center_listing(df):
    center_listing = {}
    center = df.groupby(['host_name']).apply(get_center_loc)
    for host in df.host_name.unique():
        center_listing[host] = get_group_center(df[df.host_name == host], center)
    
    return center_listing

multi_start_centers = find_center_listing(multi_start)
multi_end_centers = find_center_listing(multi_end)




In [41]:
map = folium.Map(location=get_center_loc(startDt), prefer_canvas=True, tiles="cartodb positron", zoom_start=9)

# create layers
start_group = folium.FeatureGroup(name="May 2022 Multi Owner •", control=True, show=False).add_to(map)
end_group = folium.FeatureGroup(name="May 2023 Multi Owner •", control=True, show=False).add_to(map)
start_line = folium.FeatureGroup(name="May 2022 Home •---shared-owner---• Home", control=True, show=False).add_to(map)
end_line = folium.FeatureGroup(name="May 2023 Home •---shared-owner---• Home", control=True, show=False).add_to(map)
single_start_group = folium.FeatureGroup(name="May 2022 Home • single owner", control=True, show=False).add_to(map)
single_end_group = folium.FeatureGroup(name="May 2023 Home • single owner", control=True, show=False).add_to(map)
heat_group =folium.FeatureGroup(name="% of airbnb change May 2022 to May 2023", control=True, show=False).add_to(map)

folium.LayerControl(position='bottomleft', collaped=False).add_to(map)

In [42]:
# gradient map section
# normalize weights this section is a mess and needs to be optimized
muni_weight = {}
for k, v in result_start.items():
    muni_weight[k] = v[3]

#### correct names here
muni_df = gpd.GeoDataFrame(list(muni_weight.items()), columns=['name','weight'])
geo_data = boundaries.merge(muni_df, on='name', how='left')


def weight_to_color(weight, vmin, vmax):
    cmap = plt.get_cmap('coolwarm')
    norm = mcolors.Normalize(vmin=vmin, vmax=vmax)
    rgba = cmap(norm(weight))
    return mcolors.to_hex(rgba)

# Determine the min and max weights
min_weight = geo_data['weight'].min()
max_weight = geo_data['weight'].max()

def highlight_function():
    return {'fillColor': '#000000', 
            'color':'#000000', 
            'fillOpacity': 1, 
            'weight': 0.7}


# Add GeoJson layer with popups and color mapping
folium.GeoJson(
    geo_data,
    highlight_function=lambda feature: {
        'fillColor': weight_to_color(feature['properties']['weight'], min_weight, max_weight), 
        'color':'black',
        'weight': 1
        },
    tooltip=folium.features.GeoJsonTooltip(
        fields=['name','weight'],
        aliases=['Municipality:','% change 1 year:'],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    ),
    style_function=lambda feature: {
        'fillColor': weight_to_color(feature['properties']['weight'], min_weight, max_weight),
        'weight': 0.5,
    }
).add_to(heat_group)

In [43]:
# making the lines for the connections between mulit owners
def apply_lines(x, color, group, counts, start, opacity):
    # apply lines is used for both end and start groups however we keep the same centers
    if (x.host_name in multi_end_centers):
        folium.PolyLine(
            locations=[multi_end_centers[x.host_name],[x.latitude, x.longitude]],
            popup=f'Owner {x.host_name} Mean value per night ${x.mean_price:.2f} total properties {counts[x.host_name]}',
            color=color,
            weight=1,
            opacity=opacity
        ).add_to(group) 
    else:
        folium.PolyLine(
            locations=[multi_start_centers[x.host_name],[x.latitude, x.longitude]],
            color=color,
            popup=f'Owner {x.host_name} Mean value per night ${x.mean_price:.2f} total properties {counts[x.host_name]}',
            weight=1,
            opacity=opacity
        ).add_to(group),

In [44]:
multi_start.apply(lambda x: apply_lines(x, "red", start_line, multi_start.host_name.value_counts(), True, 0.5), axis=1)

0       None
2       None
4       None
7       None
9       None
10      None
11      None
12      None
13      None
15      None
17      None
18      None
19      None
21      None
22      None
23      None
24      None
26      None
27      None
28      None
30      None
33      None
35      None
36      None
39      None
41      None
45      None
48      None
55      None
57      None
61      None
62      None
63      None
64      None
65      None
67      None
68      None
69      None
72      None
76      None
81      None
83      None
85      None
86      None
88      None
92      None
95      None
96      None
103     None
104     None
105     None
108     None
109     None
110     None
111     None
113     None
114     None
117     None
118     None
121     None
124     None
126     None
127     None
128     None
129     None
130     None
133     None
134     None
141     None
146     None
147     None
148     None
152     None
154     None
156     None
159     None
163     None

In [45]:
multi_end.apply(lambda x: apply_lines(x, "blue", end_line, multi_end.host_name.value_counts(), False, 0.5), axis=1)

1       None
2       None
5       None
7       None
8       None
9       None
10      None
11      None
13      None
14      None
15      None
17      None
18      None
19      None
20      None
21      None
22      None
23      None
25      None
27      None
29      None
30      None
32      None
33      None
38      None
39      None
46      None
48      None
51      None
52      None
53      None
54      None
55      None
56      None
57      None
58      None
61      None
64      None
67      None
71      None
72      None
74      None
78      None
81      None
87      None
90      None
92      None
94      None
95      None
97      None
98      None
99      None
103     None
105     None
106     None
107     None
108     None
109     None
111     None
112     None
117     None
122     None
123     None
124     None
129     None
131     None
134     None
139     None
141     None
143     None
146     None
150     None
151     None
153     None
155     None
159     None
166     None

In [46]:
multi_start.apply(lambda x: folium.CircleMarker(
                                location=[x.latitude, x.longitude],
                                radius=1,
                                popup=f'Price per night ${x.price}.\nHost {x.host_name}',
                                color="red",
                            ).add_to(start_group), axis=1)

0       <folium.vector_layers.CircleMarker object at 0...
2       <folium.vector_layers.CircleMarker object at 0...
4       <folium.vector_layers.CircleMarker object at 0...
7       <folium.vector_layers.CircleMarker object at 0...
9       <folium.vector_layers.CircleMarker object at 0...
10      <folium.vector_layers.CircleMarker object at 0...
11      <folium.vector_layers.CircleMarker object at 0...
12      <folium.vector_layers.CircleMarker object at 0...
13      <folium.vector_layers.CircleMarker object at 0...
15      <folium.vector_layers.CircleMarker object at 0...
17      <folium.vector_layers.CircleMarker object at 0...
18      <folium.vector_layers.CircleMarker object at 0...
19      <folium.vector_layers.CircleMarker object at 0...
21      <folium.vector_layers.CircleMarker object at 0...
22      <folium.vector_layers.CircleMarker object at 0...
23      <folium.vector_layers.CircleMarker object at 0...
24      <folium.vector_layers.CircleMarker object at 0...
26      <foliu

In [47]:
multi_end.apply(lambda x: folium.CircleMarker(
                                location=[x.latitude, x.longitude],
                                radius=.5,
                                popup=f'Price per night ${x.price}.\nHost {x.host_name}',
                                color="blue",
                            ).add_to(end_group), axis=1)

1       <folium.vector_layers.CircleMarker object at 0...
2       <folium.vector_layers.CircleMarker object at 0...
5       <folium.vector_layers.CircleMarker object at 0...
7       <folium.vector_layers.CircleMarker object at 0...
8       <folium.vector_layers.CircleMarker object at 0...
9       <folium.vector_layers.CircleMarker object at 0...
10      <folium.vector_layers.CircleMarker object at 0...
11      <folium.vector_layers.CircleMarker object at 0...
13      <folium.vector_layers.CircleMarker object at 0...
14      <folium.vector_layers.CircleMarker object at 0...
15      <folium.vector_layers.CircleMarker object at 0...
17      <folium.vector_layers.CircleMarker object at 0...
18      <folium.vector_layers.CircleMarker object at 0...
19      <folium.vector_layers.CircleMarker object at 0...
20      <folium.vector_layers.CircleMarker object at 0...
21      <folium.vector_layers.CircleMarker object at 0...
22      <folium.vector_layers.CircleMarker object at 0...
23      <foliu

In [48]:
single_start.apply(lambda x: folium.CircleMarker(
                                location=[x.latitude, x.longitude],
                                radius=.5,
                                popup=f'Price per night ${x.price}.\nHost {x.host_name}',
                                color="green",
                            ).add_to(single_start_group), axis=1)

1       <folium.vector_layers.CircleMarker object at 0...
3       <folium.vector_layers.CircleMarker object at 0...
5       <folium.vector_layers.CircleMarker object at 0...
6       <folium.vector_layers.CircleMarker object at 0...
8       <folium.vector_layers.CircleMarker object at 0...
14      <folium.vector_layers.CircleMarker object at 0...
16      <folium.vector_layers.CircleMarker object at 0...
20      <folium.vector_layers.CircleMarker object at 0...
25      <folium.vector_layers.CircleMarker object at 0...
29      <folium.vector_layers.CircleMarker object at 0...
31      <folium.vector_layers.CircleMarker object at 0...
32      <folium.vector_layers.CircleMarker object at 0...
34      <folium.vector_layers.CircleMarker object at 0...
37      <folium.vector_layers.CircleMarker object at 0...
38      <folium.vector_layers.CircleMarker object at 0...
40      <folium.vector_layers.CircleMarker object at 0...
42      <folium.vector_layers.CircleMarker object at 0...
43      <foliu

In [49]:
single_end.apply(lambda x: folium.CircleMarker(
                                location=[x.latitude, x.longitude],
                                radius=.5,
                                popup=f'Price per night ${x.price}.\nHost {x.host_name}',
                                color="black",
                            ).add_to(single_end_group), axis=1)

0       <folium.vector_layers.CircleMarker object at 0...
3       <folium.vector_layers.CircleMarker object at 0...
4       <folium.vector_layers.CircleMarker object at 0...
6       <folium.vector_layers.CircleMarker object at 0...
12      <folium.vector_layers.CircleMarker object at 0...
16      <folium.vector_layers.CircleMarker object at 0...
24      <folium.vector_layers.CircleMarker object at 0...
26      <folium.vector_layers.CircleMarker object at 0...
28      <folium.vector_layers.CircleMarker object at 0...
31      <folium.vector_layers.CircleMarker object at 0...
34      <folium.vector_layers.CircleMarker object at 0...
35      <folium.vector_layers.CircleMarker object at 0...
36      <folium.vector_layers.CircleMarker object at 0...
37      <folium.vector_layers.CircleMarker object at 0...
40      <folium.vector_layers.CircleMarker object at 0...
41      <folium.vector_layers.CircleMarker object at 0...
42      <folium.vector_layers.CircleMarker object at 0...
43      <foliu

In [50]:
# sbs = folium.plugins.SideBySideLayers(layer_left=start_group, layer_right=end_group)
# sbs.add_to(map)
legend_html = '''
<div style="position: fixed; 
     bottom: 50px; right: 50px; width: 150px; height: 110px; 
     border:2px solid grey; z-index:9999; font-size:10px;
     background-color:white;
     ">&nbsp; Legend <br>
     &nbsp; <i class="fa fa-circle" style="color:red"></i>&nbsp; May 2022 Owns Multi<br>
     &nbsp; <i class="fa fa-circle" style="color:blue"></i>&nbsp; May 2023 Owns Multi<br>
     &nbsp; <i class="fa fa-circle" style="color:green"></i>&nbsp; May 2022 Owns Single<br>
     &nbsp; <i class="fa fa-circle" style="color:black"></i>&nbsp; May 2023 Owns Single<br>
     &nbsp; <i class="fa fa-minus" style="color:red"></i>&nbsp; May 2022 Shared Owner<br>
     &nbsp; <i class="fa fa-minus" style="color:blue"></i>&nbsp; May 2023 Shared Owner<br>
</div>
'''

map.get_root().html.add_child(folium.Element(legend_html))

colormap = branca.colormap.LinearColormap(colors=['blue', 'red'],vmin=min_weight, vmax=max_weight,)
colormap.caption = '% of airbnb change'
colormap.add_to(map)

map.save('map.html')